# Accessing BLS data

There is at least one type that required some special attention in a BLS file as it was not listed in the `xAOD` EDM. To fix this, there is now a [special file](https://github.com/gordonwatts/func-adl-types-atlas/blob/main/scripts/required_classes.txt) that contains a list of `extra` types.

This demo shows how to use those classes.

## ServiceX against a dataset

We will use Peter's dataset `user.ponyisi:user.ponyisi.mc23_13p6TeV.524546.MGPy8EG_23lo_S4b18p4NJ0_Upsi1S2mu_4mu_3pt2_v2.deriv.DAOD_BPHY4.e8548_a911_r14908_p5923`.

In [1]:
dataset_name = "user.ponyisi:user.ponyisi.mc23_13p6TeV.524546.MGPy8EG_23lo_S4b18p4NJ0_Upsi1S2mu_4mu_3pt2_v2.deriv.DAOD_BPHY4.e8548_a911_r14908_p5923"
backend_name = "atlasr22"

In [2]:
from servicex import dataset

ds = dataset.Rucio(dataset_name)

## The BLS query that fetches just the orphaned type data

We have to declare the type first.

In [3]:
from typing import Iterable
from func_adl_servicex_xaodr22 import FuncADLQueryPHYSLITE
from func_adl_servicex_xaodr22.type_support import cpp_type
from func_adl_servicex_xaodr22.elementlink_datavector_xaod_muon_v1__ import (
    ElementLink_DataVector_xAOD_Muon_v1__,
)

# Define the type for std::vector<ElementLink<xAOD::MuonContainer>>
cpp_muon_links = cpp_type[Iterable[ElementLink_DataVector_xAOD_Muon_v1__]](
    "ElementLink<DataVector<xAOD::Muon_v1>>",
    Iterable[ElementLink_DataVector_xAOD_Muon_v1__],
    "std::vector<ElementLink<DataVector<xAOD::Muon_v1>>>",
)


query = (
    FuncADLQueryPHYSLITE()
    .Select(lambda e: e.Vertices("BPHY4Quads"))
    .Select(
        lambda vtxs: {
            "x": vtxs.Select(lambda v: v.x()),
            "QUAD_Muon0": vtxs.Select(
                lambda v: v.auxdataConst[cpp_muon_links]("MuonLinks")[0].pt()
            ),
        }
    )
)

Next fetch the data

In [4]:
# import logging

# logging.basicConfig(level=logging.DEBUG)

In [7]:
from servicex import deliver

request = {
    "Sample": [
        {
            "Name": "TestBLSQuadMu0",
            "Dataset": ds,
            "Query": query,
            "Codegen": "atlasr22",
            "IgnoreLocalCache": True,
        }
    ]
}

result = deliver(request, servicex_name=backend_name)

Output()

In [8]:
import uproot
f = uproot.open(result['TestBLSQuadMu0'][0])
tree = f['atlas_xaod_tree']
tree.keys()

['x', 'QUAD_Muon0']

In [9]:
tree['x'].array()

<Array [[], [], ..., [0.128, -0.938, ..., -2.88]] type='30000 * var * float32'>

In [10]:
tree['QUAD_Muon0'].array()

<Array [[], [], ..., [1.2e+04, ..., 3.43e+03]] type='30000 * var * float64'>